Save Gdrive



In [1]:
# pour monter son drive Google Drive local
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import sys
#gatien
#my_local_drive='/content/gdrive/MyDrive/M1/Projet/donnees'
#gaby
my_local_drive='/content/gdrive/My Drive/M1_IASD/Colab Notebooks/ML/Projet/donnees/'
sys.path.append(my_local_drive)


# Se positionner sur le répertoire associé
%cd $my_local_drive

%pwd
%ls

/content/gdrive/My Drive/M1_IASD/Colab Notebooks/ML/Projet/donnees
HAI817_Projet_test.csv   HAI817_Projet_train.csv  Projet_HAI817I.ipynb
HAI817_Projet_test.xlsx  MyNLPUtilities.py


## **Travaux pratiques**

Importer les différentes choses qu'on a besoin

In [76]:
# Librairie a installer

!pip install langdetect 
!pip install contractions

# Importation des différentes librairies utiles pour le notebook

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sys
import pandas as pd
import numpy as np 
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn import metrics


# librairies des classifiers utilisés
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler


# Importation des différentes librairies utiles pour le notebook ingenieurie logicielle des données textuelles

#Sickit learn met régulièrement à jour des versions et 
#indique des futurs warnings. 
#ces deux lignes permettent de ne pas les afficher.
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# librairies générales
import pickle 
from scipy.stats import randint
import string
import time
import base64
import re

import contractions

# librairie BeautifulSoup
from bs4 import BeautifulSoup

# librairie affichage
import wordcloud

## detection de language
import langdetect 

import nltk
from nltk import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from nltk import RegexpParser
# il est possible de charger l'ensemble des librairies en une seule fois 
# décocher le commentaire de la ligne ci-dessous
#nltk.download('all') 
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk import pos_tag
nltk.download('tagsets')
nltk.download("stopwords")
nltk.download('wordnet')

from nltk.corpus import stopwords

import spacy
from spacy.tokens import Span
from spacy.lang.fr import French

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package tagsets to /root/nltk_data...
[nltk_data]   Package tagsets is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


On récupère les infos du dataset

In [77]:
df=pd.read_csv('HAI817_Projet_test.csv')

Afficher le nombre de ligne et de colonnes du dataframe ainsi que les 5 premières lignes

In [78]:
print(f"Nombre de lignes : {df.shape[0]}\nNombre de colonnes : {df.shape[1]}")

df.head(5)

Nombre de lignes : 612
Nombre de colonnes : 4


,ID,text,title,our rating
0,122653045997905671927713471889615536378,The deputy secretary at the US Treasury has sa...,US Treasury deputy sec warns that shortages li...,mixture
1,275389285957305997321446227088442471741,KABUL—Approximately twelve minutes after U.S. ...,CNN Praises Taliban For Wearing Masks During A...,other
2,333248764296609831067233855420575814716,The vast majority of (oblivious) Americans hav...,Tennessee Has Just LEGALIZED Government COVID ...,false
3,264019763253447756851916399533799891538,(Natural News) There’s a secret layer of infor...,MEDICAL SHOCKER: Scientists at Sloan Kettering...,false
4,158073737187690682830899773280916034317,"A recent study reported by the NCBI, which is ...",Study Results: Facemasks are Ineffective to Bl...,false


# Pré-traitement

Copier le df dans un autre pour le pré-traitement

In [79]:
dfPret = df.copy()
#display(dfPret)

Recuperer les titres et les text

In [80]:
print("Liste des titres")
titres = dfPret["title"]
display(titres)

print("\n\nListe des textes")
textes = dfPret["text"]
display(textes)

Liste des titres


0      US Treasury deputy sec warns that shortages li...
1      CNN Praises Taliban For Wearing Masks During A...
2      Tennessee Has Just LEGALIZED Government COVID ...
3      MEDICAL SHOCKER: Scientists at Sloan Kettering...
4      Study Results: Facemasks are Ineffective to Bl...
                             ...                        
607    ‘From October 20, the Covid pass and vaccinati...
608    Bill Gates' Former Doctor Says He 'Refused To ...
609    Quebec’s expansion of COVID-19 vaccine passpor...
610    ALERT: Luc Montagnier Did NOT Say Vaccine Woul...
611    Quebec’s cannabis, liquor stores to require co...
Name: title, Length: 612, dtype: object



Liste des textes


0      The deputy secretary at the US Treasury has sa...
1      KABUL—Approximately twelve minutes after U.S. ...
2      The vast majority of (oblivious) Americans hav...
3      (Natural News) There’s a secret layer of infor...
4      A recent study reported by the NCBI, which is ...
                             ...                        
607    DISTRIBUTE TO EVERYONE who does not want to be...
608    The physician who served as Bill Gates’ privat...
609    A customer leaves an SAQ outlet on Jan. 6, the...
610    Vaccine coercion activists are using the fake ...
611    Placeholder while article actions load  Quebec...
Name: text, Length: 612, dtype: object

Supprimer les caractères spéciaux

In [81]:
print("TITRES")
for i in range(len(titres)) :
  # suppression des caractères spéciaux
  sentence = re.sub(r'[^\w\s]',' ', str(titres[i]))
  titres[i] = sentence
display(titres)
print("\n\n")

print("TEXTES")
for i in range(len(textes)) :
  # suppression des caractères spéciaux
  sentence = re.sub(r'[^\w\s]',' ', str(textes[i]))
  textes[i] = sentence
display(textes)

TITRES


0      US Treasury deputy sec warns that shortages li...
1      CNN Praises Taliban For Wearing Masks During A...
2      Tennessee Has Just LEGALIZED Government COVID ...
3      MEDICAL SHOCKER  Scientists at Sloan Kettering...
4      Study Results  Facemasks are Ineffective to Bl...
                             ...                        
607     From October 20  the Covid pass and vaccinati...
608    Bill Gates  Former Doctor Says He  Refused To ...
609    Quebec s expansion of COVID 19 vaccine passpor...
610    ALERT  Luc Montagnier Did NOT Say Vaccine Woul...
611    Quebec s cannabis  liquor stores to require co...
Name: title, Length: 612, dtype: object




TEXTES


0      The deputy secretary at the US Treasury has sa...
1      KABUL Approximately twelve minutes after U S  ...
2      The vast majority of  oblivious  Americans hav...
3       Natural News  There s a secret layer of infor...
4      A recent study reported by the NCBI  which is ...
                             ...                        
607    DISTRIBUTE TO EVERYONE who does not want to be...
608    The physician who served as Bill Gates  privat...
609    A customer leaves an SAQ outlet on Jan  6  the...
610    Vaccine coercion activists are using the fake ...
611    Placeholder while article actions load  Quebec...
Name: text, Length: 612, dtype: object

Tokenisation

In [82]:
word_tokens_list = []
for i in range(len(titres)):
    # tokenisation
    word_tokens = word_tokenize(titres[i])
    #print("Premiers tokens après nettoyage des données ", word_tokens)
    word_tokens_list.append(word_tokens)
print(word_tokens_list)

[['US', 'Treasury', 'deputy', 'sec', 'warns', 'that', 'shortages', 'likely', 'to', 'continue', 'until', 'EVERYONE', 'is', 'jabbed'], ['CNN', 'Praises', 'Taliban', 'For', 'Wearing', 'Masks', 'During', 'Attack'], ['Tennessee', 'Has', 'Just', 'LEGALIZED', 'Government', 'COVID', 'Kidnappings', 'The', 'Daily', 'Allegiant'], ['MEDICAL', 'SHOCKER', 'Scientists', 'at', 'Sloan', 'Kettering', 'discover', 'mRNA', 'inactivates', 'tumor', 'suppressing', 'proteins', 'meaning', 'it', 'can', 'promote', 'cancer'], ['Study', 'Results', 'Facemasks', 'are', 'Ineffective', 'to', 'Block', 'Transmission', 'of', 'COVID', '19', 'and', 'Actually', 'Can', 'Cause', 'Health', 'Deterioration', 'and', 'Premature', 'Death'], ['How', 'to', 'Neutralize', 'Potential', 'Damage', 'from', 'mRNA', 'Vaccines'], ['CDC', 'Quietly', 'Admits', 'The', 'Death', 'Toll', 'From', 'COVID', 'Vaccines', 'Is', 'Greater', 'Than', 'Every', 'Vaccine', 'In', 'The', 'Last', '20', 'Years', 'COMBINED'], ['EXCLUSIVE', 'Per', 'the', 'CDC', 'There

Attribuer a chaque mot un type

In [83]:
import spacy

nlp = spacy.load('en_core_web_sm')

cate_word_tokens = []

for sentence in word_tokens_list: # pour chaque phrase
    sentence_str = ' '.join(sentence) # join les elements avec un espace
    doc = nlp(sentence_str)
    pos_tags = [(token.text, token.pos_) for token in doc]
    cate_word_tokens.append(pos_tags)

print("Tokens conservés de catégorie PROPN ou NOUN ou VERB ou ADJ ou ADV ou PRON :")
print("Liste des phrases : ", word_tokens_list)
print("Liste des correspondances de type : ", cate_word_tokens)



Tokens conservés de catégorie PROPN ou NOUN ou VERB ou ADJ ou ADV ou PRON :
Liste des phrases :  [['US', 'Treasury', 'deputy', 'sec', 'warns', 'that', 'shortages', 'likely', 'to', 'continue', 'until', 'EVERYONE', 'is', 'jabbed'], ['CNN', 'Praises', 'Taliban', 'For', 'Wearing', 'Masks', 'During', 'Attack'], ['Tennessee', 'Has', 'Just', 'LEGALIZED', 'Government', 'COVID', 'Kidnappings', 'The', 'Daily', 'Allegiant'], ['MEDICAL', 'SHOCKER', 'Scientists', 'at', 'Sloan', 'Kettering', 'discover', 'mRNA', 'inactivates', 'tumor', 'suppressing', 'proteins', 'meaning', 'it', 'can', 'promote', 'cancer'], ['Study', 'Results', 'Facemasks', 'are', 'Ineffective', 'to', 'Block', 'Transmission', 'of', 'COVID', '19', 'and', 'Actually', 'Can', 'Cause', 'Health', 'Deterioration', 'and', 'Premature', 'Death'], ['How', 'to', 'Neutralize', 'Potential', 'Damage', 'from', 'mRNA', 'Vaccines'], ['CDC', 'Quietly', 'Admits', 'The', 'Death', 'Toll', 'From', 'COVID', 'Vaccines', 'Is', 'Greater', 'Than', 'Every', 'Vac

Filtrer les mots qui ne sont pas dnas la liste des categories

In [84]:
filter_cate_word = []
selective_pos = ['PROPN', 'NOUN', 'VERB', 'ADJ', 'ADV', 'PRON']
for phrase in cate_word_tokens :
  tab_courant = []
  for word in phrase :
    if word[1] in selective_pos :
      tab_courant.append(word)
  filter_cate_word.append(tab_courant)
print(filter_cate_word)

[[('US', 'PROPN'), ('Treasury', 'PROPN'), ('deputy', 'NOUN'), ('sec', 'PROPN'), ('warns', 'VERB'), ('shortages', 'NOUN'), ('likely', 'ADJ'), ('continue', 'VERB'), ('EVERYONE', 'PRON'), ('jabbed', 'NOUN')], [('CNN', 'PROPN'), ('Praises', 'VERB'), ('Taliban', 'PROPN'), ('Wearing', 'VERB'), ('Masks', 'PROPN'), ('Attack', 'PROPN')], [('Tennessee', 'PROPN'), ('Has', 'VERB'), ('Just', 'ADV'), ('LEGALIZED', 'PRON'), ('Government', 'NOUN'), ('COVID', 'VERB'), ('Kidnappings', 'PROPN'), ('Daily', 'PROPN'), ('Allegiant', 'PROPN')], [('MEDICAL', 'PROPN'), ('SHOCKER', 'VERB'), ('Scientists', 'PROPN'), ('Sloan', 'PROPN'), ('Kettering', 'PROPN'), ('discover', 'VERB'), ('mRNA', 'NOUN'), ('inactivates', 'NOUN'), ('tumor', 'VERB'), ('suppressing', 'VERB'), ('proteins', 'NOUN'), ('meaning', 'VERB'), ('it', 'PRON'), ('promote', 'VERB'), ('cancer', 'NOUN')], [('Study', 'NOUN'), ('Results', 'PROPN'), ('Facemasks', 'NOUN'), ('Ineffective', 'ADJ'), ('Block', 'PROPN'), ('Transmission', 'PROPN'), ('COVID', 'PRO

Enlever les majuscules des autres que nom propre

In [85]:
min_filter_cate = []
for phrase in filter_cate_word :
    #Mettre en minuscule
    mots_minuscules = []
    for word in phrase :
      if word[1] != 'PROPN':
          mots_minuscules.append(word[0].lower())
      else:
          mots_minuscules.append(word[0])
    min_filter_cate.append(mots_minuscules)
print(min_filter_cate)

[['US', 'Treasury', 'deputy', 'sec', 'warns', 'shortages', 'likely', 'continue', 'everyone', 'jabbed'], ['CNN', 'praises', 'Taliban', 'wearing', 'Masks', 'Attack'], ['Tennessee', 'has', 'just', 'legalized', 'government', 'covid', 'Kidnappings', 'Daily', 'Allegiant'], ['MEDICAL', 'shocker', 'Scientists', 'Sloan', 'Kettering', 'discover', 'mrna', 'inactivates', 'tumor', 'suppressing', 'proteins', 'meaning', 'it', 'promote', 'cancer'], ['study', 'Results', 'facemasks', 'ineffective', 'Block', 'Transmission', 'COVID', 'actually', 'cause', 'Health', 'Deterioration', 'Premature', 'Death'], ['neutralize', 'Potential', 'Damage', 'mRNA', 'vaccines'], ['CDC', 'quietly', 'admits', 'Death', 'Toll', 'covid', 'vaccines', 'greater', 'Vaccine', 'last', 'years', 'combined'], ['EXCLUSIVE', 'CDC', 'there', 'are', 'nearly', 'Twice', 'many', 'Vaccine', 'Related', 'deaths', 'so', 'far', 'Vaccine', 'Deaths', 'Past', 'Decade'], ['fully', 'vaccinated', 'people', 'have', 'higher', 'chance', 'death', 'Covid', 'p

Lemmatisation : retour a la forme de lemme

In [86]:
lemmatizer=WordNetLemmatizer()

lemmas_final_word = []
for phrase in min_filter_cate :
  doc_lemme = nlp(" ".join(phrase)) # joindre avec un espace
  lemmas_courant = []
  for token in doc_lemme :
    if token.text.endswith('-'): continue # enleve les mots composé avec un tiret
    lemmas_courant.append(lemmatizer.lemmatize(token.text)) # ajout a la liste en lematisant le mot
  lemmas_final_word.append(lemmas_courant) # ajout a la grande liste
print(lemmas_final_word)

[['US', 'Treasury', 'deputy', 'sec', 'warns', 'shortage', 'likely', 'continue', 'everyone', 'jabbed'], ['CNN', 'praise', 'Taliban', 'wearing', 'Masks', 'Attack'], ['Tennessee', 'ha', 'just', 'legalized', 'government', 'covid', 'Kidnappings', 'Daily', 'Allegiant'], ['MEDICAL', 'shocker', 'Scientists', 'Sloan', 'Kettering', 'discover', 'mrna', 'inactivates', 'tumor', 'suppressing', 'protein', 'meaning', 'it', 'promote', 'cancer'], ['study', 'Results', 'facemasks', 'ineffective', 'Block', 'Transmission', 'COVID', 'actually', 'cause', 'Health', 'Deterioration', 'Premature', 'Death'], ['neutralize', 'Potential', 'Damage', 'mRNA', 'vaccine'], ['CDC', 'quietly', 'admits', 'Death', 'Toll', 'covid', 'vaccine', 'greater', 'Vaccine', 'last', 'year', 'combined'], ['EXCLUSIVE', 'CDC', 'there', 'are', 'nearly', 'Twice', 'many', 'Vaccine', 'Related', 'death', 'so', 'far', 'Vaccine', 'Deaths', 'Past', 'Decade'], ['fully', 'vaccinated', 'people', 'have', 'higher', 'chance', 'death', 'Covid', 'people', 

Racinisation, mettre sous la forme générique

In [87]:
"""for word_tokens in lemmas_final_word :
  # Racinisation
  ps=nltk.stem.porter.PorterStemmer()

  print(f"Racinisation : {[ps.stem(word) for word in word_tokens]}")"""

# pour linstant on enleve pcq pas precis

'for word_tokens in lemmas_final_word :\n  # Racinisation\n  ps=nltk.stem.porter.PorterStemmer()\n\n  print(f"Racinisation : {[ps.stem(word) for word in word_tokens]}")'

Suppresion des stops worlds

In [88]:
# Affichage de la liste des stops words
nltk.download("stopwords")
from nltk.corpus import stopwords
the_stopwords=set(stopwords.words("english"))
print (the_stopwords)

{'between', "you'll", 'yourselves', "didn't", 'up', 'by', 'during', 'don', 'isn', 'her', "wouldn't", 'into', 'ours', 'more', 'didn', 'those', 'through', 'in', 'under', 'was', 'how', 'some', 'm', "couldn't", 'any', 'will', "mustn't", 'until', 'not', 'do', 'both', 'as', 'if', 'and', 'myself', 'can', "needn't", 'when', 'a', 'this', 'of', 've', 're', 'him', "you've", "you're", 'at', 'all', 'ain', 'just', 'above', 'now', 'weren', 'having', 'won', 'shan', 'than', 'herself', 'does', 'himself', "haven't", 'so', 'y', 'she', 'then', 'wasn', 'these', 'be', "hasn't", 'against', "don't", "wasn't", 'such', 'very', 'own', 'am', 'because', 'after', 'them', 'theirs', "that'll", 'were', "won't", "you'd", "should've", 'once', "hadn't", 'did', 'below', 'll', 'needn', 'itself', 'before', 'aren', 'while', 'wouldn', 'yours', 'been', 'your', 'had', 'who', 'from', 'over', 'we', 'no', 'but', 's', 'doesn', 'has', 'their', 'or', 'is', 'haven', 'what', 'an', 'only', 'each', 'few', 'it', "aren't", 'here', 'hadn', '

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [89]:
# Supression
final_list_sentence = []
for word_tokens in lemmas_final_word :
  print ("   Avant suppression des stopwords : ")
  print (word_tokens)
  tokens=[word for word in word_tokens if word not in the_stopwords]
  print ("   Après suppression des stopwords : ")
  print (tokens)
  print("\n\n")
  sentence = ' '.join(tokens)
  final_list_sentence.append(sentence)

print(final_list_sentence)



   Avant suppression des stopwords : 
['US', 'Treasury', 'deputy', 'sec', 'warns', 'shortage', 'likely', 'continue', 'everyone', 'jabbed']
   Après suppression des stopwords : 
['US', 'Treasury', 'deputy', 'sec', 'warns', 'shortage', 'likely', 'continue', 'everyone', 'jabbed']



   Avant suppression des stopwords : 
['CNN', 'praise', 'Taliban', 'wearing', 'Masks', 'Attack']
   Après suppression des stopwords : 
['CNN', 'praise', 'Taliban', 'wearing', 'Masks', 'Attack']



   Avant suppression des stopwords : 
['Tennessee', 'ha', 'just', 'legalized', 'government', 'covid', 'Kidnappings', 'Daily', 'Allegiant']
   Après suppression des stopwords : 
['Tennessee', 'ha', 'legalized', 'government', 'covid', 'Kidnappings', 'Daily', 'Allegiant']



   Avant suppression des stopwords : 
['MEDICAL', 'shocker', 'Scientists', 'Sloan', 'Kettering', 'discover', 'mrna', 'inactivates', 'tumor', 'suppressing', 'protein', 'meaning', 'it', 'promote', 'cancer']
   Après suppression des stopwords : 
['MEDI

N-grammes qui marche pas encore les lignes commentés ne marche pas

In [94]:
print("n-grammes de mots de taille 1")
vectorizer = CountVectorizer(ngram_range=(1,1))
# fit et transform en une opération
X = vectorizer.fit_transform(final_list_sentence)
#print(vectorizer.get_feature_names())

print("\nn-grammes de mots de taille 1 et 2")
vectorizer = CountVectorizer(ngram_range=(1,2))
# fit et transform en une opération
X = vectorizer.fit_transform(final_list_sentence)
#print(vectorizer.get_feature_names())

print("\nn-grammes de mots de taille 2 et 3")
vectorizer = CountVectorizer(ngram_range=(2,3))
# fit et transform en une opération
X = vectorizer.fit_transform(final_list_sentence)
#print(vectorizer.get_feature_names())

print("\nn-grammes de mots de taille 3")
vectorizer = CountVectorizer(ngram_range=(3,3))
# fit et transform en une opération
X = vectorizer.fit_transform(final_list_sentence)
#print(vectorizer.get_feature_names())

n-grammes de mots de taille 1

n-grammes de mots de taille 1 et 2


AttributeError: ignored

# Classification

Recuperer les valeurs VRAI, FAUX et AUTRE et les MIXTES, VRAI ou FAUX

In [ ]:
total_vrai = df.loc[df["our rating"] == "true"]
display(f"Nombre de lignes pour total_vrai : {total_vrai.shape[0]}")

total_faux = df.loc[df["our rating"] == "false"]
display(f"Nombre de lignes pour total_faux : {total_faux.shape[0]}")

total_vrai_ou_faux = df.loc[(df["our rating"] == "false") | (df["our rating"] == "true")]
display(f"Nombre de lignes pour total_vrai_ou_faux : {total_vrai_ou_faux.shape[0]}")

total_other = df.loc[df["our rating"] == "other"]
display(f"Nombre de lignes pour total_other : {total_other.shape[0]}")

total_mixte = df.loc[df["our rating"] == "mixture"]
display(f"Nombre de lignes pour total_mixture : {total_mixte.shape[0]}")

Existe-t'il des valeurs nulles ? Existe-til des valeurs manquantes ? Rappel vous pouvez le voir avec des histogrammes mais aussi avec une heatmap.  